In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# -- 1) EXACT SOLUTION FOR THE OSCILLATOR --
def damped_harmonic_oscillator(t, m=1.0, mu=0.6, k=5.0, y0=-0.4, v0=3.0):
    # Check underdamped condition
    if mu**2 >= 4 * m * k:
        raise ValueError("The system is not underdamped. Ensure μ^2 < 4 * m * k.")
    
    omega = np.sqrt(k/m - (mu/(2*m))**2)
    
    A = y0
    B = (v0 + (mu/(2*m)) * y0) / omega
    y_exact = np.exp(-mu * t / (2*m)) * (A * np.cos(omega * t) + B * np.sin(omega * t))
    return y_exact


# -- 2) PDF FUNCTIONS FOR p(x) AND g(x) --
def p_pdf(m, mu, k):
    """ PDF of the ORIGINAL distribution (product of independent normals).
        m ~ N(1, 0.1^2), mu ~ N(0.6, 0.05^2), k ~ N(5, 0.2^2), y0 ~ N(-0.4, 0^2)
    """
    pm = norm.pdf(m, loc=1.0, scale=0.1)
    pmu = norm.pdf(mu, loc=0.6, scale=0.05)
    pk = norm.pdf(k, loc=5.0, scale=0.2)
    return pm * pmu * pk

def g_pdf(m, mu, k):
    """ PDF of the PROPOSAL distribution g(x).
        Adjust means or std to focus on "dangerous" region, e.g. larger mass or smaller damping
    """
    # Example: shift mass to a higher mean (like 1.3) and damping to a smaller mean (like 0.45)
    gm = norm.pdf(m, loc=1.3, scale=0.2)
    gmu = norm.pdf(mu, loc=0.45, scale=0.05)
    gk = norm.pdf(k, loc=5.0, scale=0.3)
    return gm * gmu * gk

def sample_from_g(N):
    """ Draw N samples from the proposal distribution g(x). """
    m_samples = np.random.normal(loc=1.8, scale=0.2, size=N)
    mu_samples = np.random.normal(loc=0.5, scale=0.05, size=N)
    k_samples = np.random.normal(loc=5.0, scale=0.3, size=N)
    return m_samples, mu_samples, k_samples


# -- 3) IMPORTANCE SAMPLING --
def IS_sim(N, t, threshold=-1.0):
    """
    Estimate probability that oscillator crosses below 'threshold'
    at ANY time in [t.min(), t.max()] using importance sampling.
    """
    # 1) Sample from g(x)
    m_samples, mu_samples, k_samples = sample_from_g(N)

    # 2) Evaluate weights w_i = p(x_i) / g(x_i)
    w = []
    indicators = []
    for i in range(N):
        # Evaluate ratio
        px = p_pdf(m_samples[i], mu_samples[i], k_samples[i] )
        gx = g_pdf(m_samples[i], mu_samples[i], k_samples[i] )
        w.append(px / gx if gx > 1e-30 else 0.0)

        y0_samples = np.full(N, -0.4)

    # 3) Evaluate failure: does y(t) < threshold at ANY time?
        y = damped_harmonic_oscillator(t, m_samples[i], mu_samples[i],
                                       k_samples[i], y0_samples[i], v0=3.0)
        # if any point is below threshold, it's a 'failure'
        fail = np.any(y < threshold)
        indicators.append(fail)

    w = np.array(w)
    indicators = np.array(indicators, dtype=float)

    # 4) Weighted average for probability
    # p_fail = (1/N) * sum( I{fail} * w_i )
    p_fail = np.mean(indicators * w)

    return p_fail

# 4) Run and compare with standard MC
if __name__ == "__main__":
    # np.random.seed(0)
    t = np.linspace(0, 5, 100)
    N = 5000

    # Importance sampling approach
    p_fail_is = IS_sim(N, t, threshold=-0.9)

    print(f"Importance sampling estimate: {p_fail_is}")


Importance sampling estimate: 0.0012332267814176184


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Exact solution for the damped oscillator
def damped_harmonic_oscillator(t, m=1.0, mu=0.6, k=5.0, y0=-0.4, v0=3.0):
    if mu**2 >= 4 * m * k:
        raise ValueError("The system is not underdamped. Ensure μ^2 < 4*m*k.")
    omega = np.sqrt(k/m - (mu/(2*m))**2)
    A = y0
    B = (v0 + (mu/(2*m)) * y0) / omega
    y_exact = np.exp(-mu * t / (2*m)) * (A * np.cos(omega * t) + B * np.sin(omega * t))
    return y_exact

# Original target distribution p(x)
def p_pdf(m, mu, k, y0):
    pm = norm.pdf(m, loc=1.0, scale=0.1)
    pmu = norm.pdf(mu, loc=0.6, scale=0.05)
    pk = norm.pdf(k, loc=5.0, scale=0.2)
    py0 = 1.0  # y0 is fixed at -0.4
    return pm * pmu * pk * py0

# Initial proposal distribution g(x)
def sample_from_g_initial(N):
    # Choose a broad distribution to cover a wide range
    m_samples = np.random.normal(loc=1.2, scale=0.2, size=N)
    mu_samples = np.random.normal(loc=0.5, scale=0.05, size=N)
    k_samples = np.random.normal(loc=5.0, scale=0.3, size=N)
    y0_samples = np.full(N, -0.4)
    return m_samples, mu_samples, k_samples, y0_samples

def g_pdf_initial(m, mu, k, y0):
    gm = norm.pdf(m, loc=1.2, scale=0.2)
    gmu = norm.pdf(mu, loc=0.5, scale=0.05)
    gk = norm.pdf(k, loc=5.0, scale=0.3)
    gy0 = 1.0
    return gm * gmu * gk * gy0

# Adaptive Importance Sampling
def adaptive_IS_sim(N, t, threshold=-1.0):
    # ------ Round 1: Initial Proposal ------
    m_samples, mu_samples, k_samples, y0_samples = sample_from_g_initial(N)
    weights_round1 = []
    indicators_round1 = []
    
    for i in range(N):
        px = p_pdf(m_samples[i], mu_samples[i], k_samples[i], y0_samples[i])
        gx = g_pdf_initial(m_samples[i], mu_samples[i], k_samples[i], y0_samples[i])
        weight = px / gx if gx > 1e-30 else 0.0
        weights_round1.append(weight)
        y = damped_harmonic_oscillator(t, m_samples[i], mu_samples[i], k_samples[i], y0_samples[i], v0=3.0)
        fail = np.any(y < threshold)
        indicators_round1.append(fail)
    
    weights_round1 = np.array(weights_round1)
    indicators_round1 = np.array(indicators_round1, dtype=float)
    p_fail_round1 = np.mean(indicators_round1 * weights_round1)
    print("Round 1 IS estimate:", p_fail_round1)
    
    # Identify failure samples from round 1
    failure_indices = np.where(indicators_round1 == 1)[0]
    print("Number of failure samples in Round 1:", len(failure_indices))
    if len(failure_indices) == 0:
        print("No failures in Round 1; cannot adapt proposal.")
        return p_fail_round1
    
    # ------ Fit Adaptive Proposal based on failure samples ------
    m_fail = m_samples[failure_indices]
    mu_fail = mu_samples[failure_indices]
    k_fail = k_samples[failure_indices]
    
    # Compute the mean and std for failure samples
    m_mean, m_std = np.mean(m_fail), np.std(m_fail)
    mu_mean, mu_std = np.mean(mu_fail), np.std(mu_fail)
    k_mean, k_std = np.mean(k_fail), np.std(k_fail)
    
    print("Adaptive proposal parameters:")
    print("m: mean =", m_mean, ", std =", m_std)
    print("mu: mean =", mu_mean, ", std =", mu_std)
    print("k: mean =", k_mean, ", std =", k_std)
    
    # Define adaptive proposal sampling functions using the fitted parameters
    def sample_from_g_adaptive(N):
        m_samples = np.random.normal(loc=m_mean, scale=m_std, size=N)
        mu_samples = np.random.normal(loc=mu_mean, scale=mu_std, size=N)
        k_samples = np.random.normal(loc=k_mean, scale=k_std, size=N)
        y0_samples = np.full(N, -0.4)
        return m_samples, mu_samples, k_samples, y0_samples
    
    def g_pdf_adaptive(m, mu, k, y0):
        gm = norm.pdf(m, loc=m_mean, scale=m_std)
        gmu = norm.pdf(mu, loc=mu_mean, scale=mu_std)
        gk = norm.pdf(k, loc=k_mean, scale=k_std)
        gy0 = 1.0
        return gm * gmu * gk * gy0
    
    # ------ Round 2: Adaptive Proposal ------
    m_samples2, mu_samples2, k_samples2, y0_samples2 = sample_from_g_adaptive(N)
    weights_round2 = []
    indicators_round2 = []
    
    for i in range(N):
        px = p_pdf(m_samples2[i], mu_samples2[i], k_samples2[i], y0_samples2[i])
        gx = g_pdf_adaptive(m_samples2[i], mu_samples2[i], k_samples2[i], y0_samples2[i])
        weight = px / gx if gx > 1e-30 else 0.0
        weights_round2.append(weight)
        y = damped_harmonic_oscillator(t, m_samples2[i], mu_samples2[i], k_samples2[i], y0_samples2[i], v0=3.0)
        fail = np.any(y < threshold)
        indicators_round2.append(fail)
    
    weights_round2 = np.array(weights_round2)
    indicators_round2 = np.array(indicators_round2, dtype=float)
    p_fail_round2 = np.mean(indicators_round2 * weights_round2)
    print("Round 2 (adaptive) IS estimate:", p_fail_round2)
    
    return p_fail_round2

# Run the adaptive IS simulation
if __name__ == "__main__":
    #np.random.seed(0)
    t = np.linspace(0, 5, 100)
    N = 10000
    p_fail_adaptive = adaptive_IS_sim(N, t, threshold=-0.696)
    print("Final adaptive IS probability estimate:", p_fail_adaptive)


Round 1 IS estimate: 0.48752173893245365
Number of failure samples in Round 1: 9485
Adaptive proposal parameters:
m: mean = 1.2225648380666951 , std = 0.1787489353499321
mu: mean = 0.4980809618405072 , std = 0.049974190514808
k: mean = 4.997952415664895 , std = 0.2994074464119004
Round 2 (adaptive) IS estimate: 0.4667789632012057
Final adaptive IS probability estimate: 0.4667789632012057
